# Состав команды
### Ячменьков Алексей:
#### 
#####
### Баранов Владислав:
#### 
#####
### Аллабердин Богдан:
#### 

In [4]:
import os
from pathlib import Path

import cv2
import wandb
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

# create train test

In [3]:
################################

In [ ]:
def extract_sample(n_way, n_support, n_query, datax, datay):
    transorm = transforms.Compose([
        np.float32,
        transforms.ToTensor(),
        fixed_image_standardization
    ])
    
    sample = []
    K = np.random.choice(np.unique(datay), n_way, replace=False)
    for cls in K:
        datax_cls = datax[datay == cls]
        perm = np.random.permutation(datax_cls)
        sample_cls = perm[:(n_support + n_query)]
        sample.append([transform(Image.open(fname)) for fname in sample_cls])
        #sample.append([cv2.imread(fname) for fname in sample_cls])

    sample = np.array(sample)
    sample = torch.from_numpy(sample).float()
    sample = sample.permute(0, 1, 4, 2, 3)
    return ({
        'images': sample,
        'targets': K,
        'n_way': n_way,
        'n_support': n_support,
        'n_query': n_query
    })

In [ ]:
def display_sample(sample):
    sample_4D = sample.view(sample.shape[0] * sample.shape[1], *sample.shape[2:])
    out = torchvision.utils.make_grid(sample_4D, nrow=sample.shape[1])

    plt.figure(figsize=(16, 7))
    plt.imshow(out.permute(1, 2, 0))

In [ ]:
sample_example = extract_sample(8, 5, 5, trainx, trainy)
display_sample(sample_example['images'])
print(sample_example['targets'])

In [ ]:
sample_example['images'].shape

## Build model

In [ ]:
class ProtoNet(nn.Module):
    def __init__(self, device='cuda'):
        super(ProtoNet, self).__init__()
        self.device = device
        self.encoder = InceptionResnetV1(pretrained='vggface2').to(self.device)

    def set_forward_loss(self, sample):
        sample_images = sample['images']
        n_way = sample['n_way']
        n_support = sample['n_support']
        n_query = sample['n_query']

        sample_images = sample_images.to(self.device)
        sample_images = sample_images.view(n_way * (n_support + n_query), *sample_images.shape[-3:])

        # img2vec results
        vectors = self.encoder(sample_images).view(n_way, (n_support + n_query), -1)

        # центры тяжести класстеров классов
        prototypes = vectors[:, :n_support].mean(1)

        # ищем расстояния от каждого изображения из query до каждого центра тяжести
        queries = vectors[:, n_support:].contiguous().view(-1, vectors.shape[-1])
        query_dists = list()
        for query in queries:
            prototype_dists = torch.stack([torch.sqrt(torch.pow(query - prototype, 2).sum()) for prototype in prototypes])
            query_dists.append(prototype_dists)
        query_dists = torch.stack(query_dists)

        probabilities = F.log_softmax(-query_dists, dim=1).view(n_way, n_query, -1)

        _, y_hat = probabilities.max(2)

        losses = list()
        n_true_positive = 0
        for way in range(n_way):
            for query in range(n_query):
                losses.append(-probabilities[way][query][way])
                if y_hat[way][query] == way:
                    n_true_positive += 1
        loss_val = torch.stack(losses).mean()
        acc = n_true_positive / (n_way * n_query)

        return loss_val, {
            'loss': loss_val.item(),
            'acc': acc,
            'y_hat': y_hat
            }

## Train

In [ ]:
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size):
    if wandb.run is not None:
        wandb.watch(model)
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
    epoch = 0 
    stop = False 

    train_losses = []
    train_acc = []
    while epoch < max_epoch and not stop:
        running_loss = 0.0
        running_acc = 0.0

        for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):
            sample = extract_sample(n_way, n_support, n_query, train_x, train_y)
            optimizer.zero_grad()
            loss, output = model.set_forward_loss(sample)
            running_loss += output['loss']
            running_acc += output['acc']
            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / epoch_size
        epoch_acc = running_acc / epoch_size
        
        train_losses.append(epoch_loss)
        train_acc.append(epoch_acc)
        
        if wandb.run is not None:
            wandb.log({'Epoch': epoch + 1, 'Loss': round(epoch_loss, 4), 'Acc': round(epoch_acc, 4)})
        print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch+1,epoch_loss, epoch_acc))
        epoch += 1
        scheduler.step()
    torch.save(model.state_dict(), "new_model.pt")
    return train_losses, train_acc

In [ ]:
should_log = False

model = ProtoNet(device='cuda')

optimizer = optim.Adam(model.parameters(), lr=0.001)

n_way = 20
n_support = 5
n_query = 5

train_x = trainx
train_y = trainy

max_epoch = 3
epoch_size = 1000

In [ ]:
from show_train_res import plot_losses

In [ ]:
%%time
if should_log:
    wandb.init(project='homework', name='baseline')
loss, acc = train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size)

## Графики обучения ProtoNet

In [ ]:
plot_losses(np.array([loss, acc]), np.array([[],[]]), 'loss', 'acc')

## Test

In [ ]:
def test(model, test_x, test_y, n_way, n_support, n_query, test_episode):
    if wandb.run is not None:
        wandb.watch(model)
    running_loss = 0.0
    running_acc = 0.0
    for episode in tnrange(test_episode):
        sample = extract_sample(n_way, n_support, n_query, test_x, test_y)
        loss, output = model.set_forward_loss(sample)
        running_loss += output['loss']
        running_acc += output['acc']

    avg_loss = running_loss / test_episode
    avg_acc = running_acc / test_episode
    if wandb.run is not None:
        wandb.log({'n_way': n_way, 'k_shot': n_support})
        wandb.log({'Test loss': round(avg_loss, 4), 'Test acc': round(avg_acc, 4), 'n_way': n_way, 'k_shot': n_support})
    return [avg_loss, avg_acc]

In [ ]:
n_way = 5
n_support = 1
n_query = 5

test_x = testx
test_y = testy

test_episode = 1000

model = ProtoNet(device='cuda')
model.load_state_dict(torch.load('./new_model.pt'))

In [ ]:
print('test: 5-way 1-shot')
output = test(model, test_x, test_y, 5, 1, n_query, test_episode)
print(f'loss: {output[0]}, acc: {output[1]}')
if wandb.run is not None:
    wandb.finish()

## Prediction on our images

In [ ]:
def predict(model, sample):
    with torch.inference_mode():
        output = model.set_forward_loss(sample)[1]['y_hat'].to('cpu').detach().numpy()
        preds = sample['targets'][output]
        print('predict : target')
        for pred, target in zip(preds, sample['targets']):
            print(f'{pred[0]} : {target}')